In [ ]:
from transformers import (
    Trainer,
    TrainingArguments,
    OPTForCausalLM,
)
from zo2 import (
    ZOConfig,
    zo2_hf_init,
)

In [ ]:
# ZO steps
zo_config = ZOConfig(method="mezo-sgd")
with zo2_hf_init(zo_config):
    model = OPTForCausalLM.from_pretrained(checkpoint)

In [ ]:
# normal trainer steps
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()